In [1]:
import pandas as pd
from keras.models import load_model
from sklearn.model_selection import train_test_split
from keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from keras.models import Model
import matplotlib.pyplot as plt
%matplotlib inline
import pickle

Using TensorFlow backend.


In [2]:
reviews_dataset = pd.read_pickle('reviews.pkl')

In [3]:
reviews_dataset.shape

(103930, 5)

In [4]:
reviews_dataset.head(5)

,isbn,user_id,user_name,user_ratings,user_review_text
0,9780321948540,779608,Erin,4.0,"There are a lot of great tips in this volume, ..."
1,9780321948540,753824,Rolf Häsänen,4.0,Not as many useful tips as volume1 unless you ...
2,9780321948540,35798331,Valery,4.0,"Not as helpful as the first, but does go into ..."
3,9780321948540,2100772,Bruce,1.0,"OMG. Surely the absolute worst, amature photo ..."
4,9780321948540,1794100,Icepick,5.0,"The format may not be for everybody, but I rea..."


In [5]:
reviews_dataset.drop(columns=['user_name','user_review_text'],inplace=True)

In [6]:
reviews_dataset.isnull().sum()

isbn                0
user_id         55729
user_ratings    55729
dtype: int64

In [7]:
reviews_dataset.dropna(inplace=True)

In [8]:
reviews_dataset.shape

(48201, 3)

In [9]:
reviews_dataset.head(5)

,isbn,user_id,user_ratings
0,9780321948540,779608,4.0
1,9780321948540,753824,4.0
2,9780321948540,35798331,4.0
3,9780321948540,2100772,1.0
4,9780321948540,1794100,5.0


In [10]:
uniq_isbn = reviews_dataset.isbn.unique()

In [11]:
isbn_dict = {uniq_isbn[i] : i+1 for i in range(0, len(uniq_isbn) ) }

In [12]:
reviews_dataset.replace({"isbn": isbn_dict},inplace=True) 

In [13]:
uniq_user = reviews_dataset.user_id.unique()

In [14]:
user_dict = {uniq_user[i] : i+1 for i in range(0, len(uniq_user) ) }

In [15]:
reviews_dataset.replace({"user_id": user_dict},inplace=True) 

In [16]:
reviews_dataset.head()

,isbn,user_id,user_ratings
0,1,1,4.0
1,1,2,4.0
2,1,3,4.0
3,1,4,1.0
4,1,5,5.0


In [17]:
train_data, test_data = train_test_split(reviews_dataset, test_size=0.2, random_state=42)

In [18]:
train_data.head()

,isbn,user_id,user_ratings
44027,1130,14273,5.0
73502,1904,22318,5.0
29611,719,8553,5.0
13919,306,3427,4.0
18143,398,4203,4.0


In [19]:
test_data.head()

,isbn,user_id,user_ratings
26190,611,7006,3.0
60771,1556,8022,2.0
42023,1069,13111,4.0
7442,155,1240,1.0
93967,2387,27240,5.0


In [20]:
#Build keras model
books_shape = Input(shape=[1])
books_data = Embedding(len(reviews_dataset.isbn.unique())+1, 5)(books_shape)
books = Flatten()(books_data)
users_shape = Input(shape=[1])
users_data = Embedding(len(reviews_dataset.user_id.unique())+1, 5)(users_shape)
users = Flatten()(users_data)
books_users = Concatenate()([books, users])
books_conn_user = Dense(64, activation='relu')(books_users)
rating = Dense(1)(books_conn_user)
deepmodel = Model([users_shape, books_shape], rating)
deepmodel.compile('adam', 'mean_squared_error',metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [21]:
deepmodel.fit([train_data.user_id, train_data.isbn], train_data.user_ratings, epochs=75, verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/75
38560/38560 [==============================] - 8s 200us/step - loss: 2.7457 - acc: 0.2688
Epoch 2/75
38560/38560 [==============================] - 7s 189us/step - loss: 0.9897 - acc: 0.4613
Epoch 3/75
38560/38560 [==============================] - ETA: 0s - loss: 0.5595 - acc: 0.629 - 7s 191us/step - loss: 0.5596 - acc: 0.6291
Epoch 4/75
38560/38560 [==============================] - 7s 186us/step - loss: 0.4238 - acc: 0.7141
Epoch 5/75
38560/38560 [==============================] - 7s 192us/step - loss: 0.3684 - acc: 0.7515
Epoch 6/75
38560/38560 [==============================] - 7s 188us/step - loss: 0.3436 - acc: 0.7670
Epoch 7/75
38560/38560 [==============================] - 8s 199us/step - loss: 0.3247 - acc: 0.7799
Epoch 8/75
38560/38560 [==============================] - 7s 187us/step - loss: 0.3097 - acc: 0.7867
Epoch 9/75
38560/38560 [==============================] - 8s 204us/step - loss: 0.2941 - acc: 0.7953
Epoch

In [22]:
deepmodel.evaluate([test_data.user_id, test_data.isbn], test_data.user_ratings)

9641/9641 [==============================] - 0s 29us/step


[1.7863750575044048, 0.40151436574824334]

In [23]:
all_isbn_list = pickle.load(open("all_isbn_list.pkl","rb"))

In [24]:
reviews_dataset.head(10)

,isbn,user_id,user_ratings
0,1,1,4.0
1,1,2,4.0
2,1,3,4.0
3,1,4,1.0
4,1,5,5.0
5,1,6,4.0
6,1,7,4.0
7,1,8,4.0
8,1,9,0.0
9,1,10,4.0


In [25]:
deepmodel.save('deeplenarningmodel')

In [26]:
model = load_model('deeplenarningmodel')

In [27]:
isbn_title_dict = pickle.load(open("isbn_title_dict.pkl","rb"))

In [28]:
usersbooks = pickle.load(open("user_books_sorted.pkl","rb"))

In [29]:
isbns = dict((v,k) for k,v in isbn_dict.items())

In [30]:
def top10deep(user): 
    bookratings = []
    for book in isbns:
        ud = user_dict[user]
        if user not in usersbooks.keys():
            continue
        x = pd.Series([ud])
        y = pd.Series([book])
        predictions = model.predict([x, y])
        bookratings.append([predictions[0][0],isbns[book]])
    
    bookratings = [item for item in bookratings if item[1] in isbn_title_dict.keys()]
    bookratings = [item for item in bookratings if item[1] not in usersbooks[user]]   
    bookratings.sort(key=lambda x: x[0], reverse=True)
    return bookratings[:10]

In [31]:
#get isbn from recommended ratings
def top10isbn(recommendations):
    top10_isbn = []
    for item in recommendations:
        item_isbn = item[1]
        top10_isbn.append(item_isbn)
    return top10_isbn

In [32]:
deeplrecommended = top10deep('269235')
isbn_recommendations = top10isbn(deeplrecommended)
print("Deep learning based model recommendations books : ")
for isbnitem in isbn_recommendations :
    print("ISBN :", str(isbnitem), "Title :", str(isbn_title_dict[isbnitem]))

Deep learning based model recommendations books : 
ISBN : 9781453237915 Title : West with the Night
ISBN : 9781599908755 Title : Starcross
ISBN : 9781596431317 Title : Drawing Words and Writing Pictures
ISBN : 9780143111597 Title : The Left Hand of Darkness
ISBN : 9782080201331 Title : Master Photographers
ISBN : 9780385372091 Title : Green Eggs and Ham: Read & Listen Edition
ISBN : 9780757305603 Title : The Sleepeasy Solution
ISBN : 9780821215517 Title : Examples
ISBN : 9780936861579 Title : Captives of Blue Mountain
ISBN : 9780805800586 Title : The Psychology of Learning Mathematics
